In [1]:
import pandas as pd
import numpy as np
import datetime
import pytz

#pd.read_csv('jiradataset/issues.csv')

df = pd.read_csv('jiradataset/issues.csv')
ch = pd.read_csv('jiradataset/changelog.csv')
sp = pd.read_csv('jiradataset/sprints.csv')

df['fields.created'] = pd.to_datetime(df['fields.created'], utc=True)
df['fields.updated'] = pd.to_datetime(df['fields.updated'], utc=True)
df['fields.resolutiondate'] = pd.to_datetime(df['fields.resolutiondate'], utc=True)

ch['created'] = pd.to_datetime(ch['created'], utc=True)
sp['sprint.completeDate'] = pd.to_datetime(sp['sprint.completeDate'], errors='ignore', utc=True)
sp['sprint.endDate'] = pd.to_datetime(sp['sprint.endDate'], errors='ignore', utc=True)
sp['sprint.startDate'] = pd.to_datetime(sp['sprint.startDate'], errors='ignore', utc=True)

# Data fixes

- Sprint startDate must be fixed because it is incomplete from the extraction. startDate is consider as the day after the endDate of the previous sprint. 

Extract the sprint number

In [2]:
# is it necessary?
# sp['sprint.number'] = sp['sprint.name'].apply(lambda x : x.split()[-1])

Fix start date:

In [3]:
# apstud - startDate must be fixed
# timob - startDate must be fixed
# tistud - startDate must be fixed

In [4]:
# each projects must be processed separately
sprint_list = []
for p in ['apstud', 'timob', 'tistud']:
    sp0 = sp[ sp.project == p ]
    sprints = sp0.groupby(['project', 'sprint.name', 'sprint.endDate'])['key'].count()
    sprints = sprints.reset_index()
    sprints = sprints.sort_values(['project','sprint.endDate'], ascending=True)
    sprints['startDate'] = sprints.shift(1)['sprint.endDate']
    
    # get the average sprint length
    sprints['startDate'] = pd.to_datetime(sprints['startDate'], utc=True)
    sprints['sprint.endDate'] = pd.to_datetime(sprints['sprint.endDate'], utc=True)
    print(p, np.mean(sprints['sprint.endDate'] - sprints['startDate']))
    
    # I will substract an average sprint lenght 
    sprints.loc[sprints['startDate'].isnull(), 'startDate'] = sprints.loc[sprints['startDate'].isnull(), 'sprint.endDate'] - np.mean(sprints['sprint.endDate'] - sprints['startDate'])
    
    sprint_list.append(sprints)
    
sprints = pd.concat(sprint_list, axis=0)

apstud 25 days 05:05:27.272727
timob 8 days 02:30:51.428571
tistud 14 days 00:49:39.310344


In [5]:
sprints[sprints['startDate'].isnull()]

,project,sprint.name,sprint.endDate,key,startDate


In [6]:
# full dataset (issues + sprints)

In [7]:
df.columns

Index(['key', 'project', 'fields.assignee.name', 'fields.created',
       'fields.updated', 'fields.resolutiondate', 'fields.status.name',
       'fields.issuetype.name', 'storypoints'],
      dtype='object')

In [8]:
# create a full dataset
full = sp.merge(df[['project', 'key', 'fields.updated', 'fields.resolutiondate', 'storypoints', 'fields.status.name', 'fields.assignee.name']], how='left', on=['project', 'key'] )

In [9]:
# fix the sprints info that is wrong
for j, v in sprints.iterrows():
    full.loc[ ((full['project'] == v['project']) & (full['sprint.name'] == v['sprint.name'])), 'sprint.startDate'] = v['startDate']

# sanity checks

In [10]:
# list of sprints
sprints = full.groupby(['project', 'sprint.name', 'sprint.startDate', 'sprint.endDate'], as_index=False)['key'].count()

In [11]:
sprints = sprints[(sprints['sprint.startDate'] != '<null>') | (sprints['sprint.endDate'] != '<null>') ]

In [12]:
sprints['sprint.startDate'] = pd.to_datetime(sprints['sprint.startDate'], utc=True)
sprints['sprint.endDate'] = pd.to_datetime(sprints['sprint.endDate'], utc=True)

sprints = sprints.sort_values(['project', 'sprint.endDate'], ascending=True)

In [13]:
sprints['sprint.length'] = sprints['sprint.endDate'] - sprints['sprint.startDate']

In [14]:
assert np.all(sprints['sprint.length'] >= pd.Timedelta(0,'D')), "there are negative sprint length"

In [15]:
# save the curated list of sprints
sprints.to_csv('data/sprints_curated.csv', index=False)

# cleaning

* incomplete data
* unreasonable storypoints (i.e. 0)

In [16]:
sp = sp[(sp['sprint.startDate'] != '<null>') | (sp['sprint.endDate'] != '<null>') ]

In [17]:
df['fields.issuetype.name'].unique()

array(['Improvement', 'Bug', 'Story', 'Technical task', 'Epic', 'Task',
       'New Feature', 'Sub-task', 'Documentation', 'Wish',
       'Patch submission', 'Technical Debt'], dtype=object)

In [18]:
#full = full.dropna()
full = full.dropna(subset=['key', 'project', 'sprint.endDate', 'sprint.name', 'sprint.startDate', 'storypoints'])

In [19]:
full['storypoints'].unique()

array([ 1. ,  3. ,  5. ,  2. ,  0. ,  8. ,  4. , 24. , 12. ,  7. ,  0.2,
        6. , 10. , 20. , 16. , 40. , 15. , 14. , 13. ,  1.5,  2.5,  1.3,
        1.2,  0.1,  1.7,  5.4,  2.4,  1.1,  1.4,  3.3, 89. ,  0.5, 21. ,
       17. , 19. , 30. , 34. ])

In [20]:
full = full[full['storypoints'] != 0]

In [21]:
full = full[full['key'].isin( df[df['fields.issuetype.name'].isin(['Task', 'Story', 'Sub-task', 'Bug', 'Technical task', 'Improvement'])]['key'] )]

In [22]:
df['fields.issuetype.name'].unique()

array(['Improvement', 'Bug', 'Story', 'Technical task', 'Epic', 'Task',
       'New Feature', 'Sub-task', 'Documentation', 'Wish',
       'Patch submission', 'Technical Debt'], dtype=object)

In [23]:
full['sprint.completeDate'] = pd.to_datetime(full['sprint.completeDate'], errors='coerce', utc=True)
full['sprint.endDate'] = pd.to_datetime(full['sprint.endDate'], errors='coerce', utc=True)
full['sprint.startDate'] = pd.to_datetime(full['sprint.startDate'], errors='coerce', utc=True)
full['fields.updated'] = pd.to_datetime(full['fields.updated'], errors='coerce', utc=True)
full['fields.resolutiondate'] = pd.to_datetime(full['fields.resolutiondate'], errors='coerce', utc=True)

In [24]:
df['fields.status.name'].unique()

array(['To Do', 'In PR', 'Done', 'In Progress', 'Closed', 'Open',
       'Reopened', 'Pull Request Submitted', 'Resolved',
       'Planned Development', 'In  Review', 'Reviewable', 'Accepted',
       'Refine', 'Raw'], dtype=object)

In [25]:
df.groupby('project')['fields.status.name'].value_counts()

project  fields.status.name    
apstud   Closed                     618
         Open                       228
         Resolved                    23
         Reopened                    16
         In  Review                   1
dnn      Closed                    3177
         Open                        96
         Reopened                    40
         Pull Request Submitted       8
         Resolved                     4
         Planned Development          3
mesos    Resolved                  1876
         Accepted                   186
         Reviewable                 125
         Open                        66
         In Progress                 51
mule     Closed                    1390
         To Do                       94
         In Progress                  7
         Resolved                     4
         Reopened                     2
nexus    Closed                    1176
         Done                        48
         Open                        42
        

In [26]:
DONE = ['Accepted', 'Closed', 'Done', 'Resolved']
DOING = ['In Progress', 'Pull Request Submitted', 'Reopened', 'Reviewable']
TODO = ['Open', 'To Do']

In [27]:
full['key'].nunique()

8807

# last status "closed"

Closed is the last status of all the projects but XD and MESOS.
Done is the last status of XD
Resolved is the last status of MESOS

In [28]:
## all the projects but XD and MESOS
ch0 = ch[~ch['project'].isin(['xd', 'mesos'])]
ch0 = ch0[ch0['toString'] == 'Closed']
ch0 = ch0[ch0['fromString'] != ch0['toString']]

closed0 = ch0.groupby(['project', 'key'], as_index=False)['created'].last()
closed0.columns = ['project', 'key', 'closed_date']

## XD
ch1 = ch[ch['project'].isin(['xd'])]
ch1 = ch1[ch1['toString'] == 'Done']
ch1 = ch1[ch1['fromString'] != ch1['toString']]

closed1 = ch1.groupby(['project', 'key'], as_index=False)['created'].last()
closed1.columns = ['project', 'key', 'closed_date']

# MESOS
ch2 = ch[ch['project'].isin(['mesos'])]
ch2 = ch2[ch2['toString'] == 'Resolved']
ch2 = ch2[ch2['fromString'] != ch2['toString']]

closed2 = ch2.groupby(['project', 'key'], as_index=False)['created'].last()
closed2.columns = ['project', 'key', 'closed_date']

# join results
closed = pd.concat([closed0, closed1, closed2], axis=0)
closed

,project,key,closed_date
0,apstud,APSTUD-1313,2012-04-20 14:47:57+00:00
1,apstud,APSTUD-1378,2012-03-27 15:49:56+00:00
2,apstud,APSTUD-1469,2012-08-15 23:14:49+00:00
3,apstud,APSTUD-1507,2012-05-20 21:43:47+00:00
4,apstud,APSTUD-1698,2012-07-13 10:42:31+00:00
...,...,...,...
1182,mesos,MESOS-830,2015-08-16 17:55:52.519000+00:00
1183,mesos,MESOS-920,2016-01-25 06:05:34.799000+00:00
1184,mesos,MESOS-934,2015-12-22 21:04:38.214000+00:00
1185,mesos,MESOS-988,2016-01-14 21:44:25.275000+00:00


In [29]:
closed.project.unique()

array(['apstud', 'dnn', 'mule', 'nexus', 'timob', 'tistud', 'xd', 'mesos'],
      dtype=object)

In [30]:
full = full.merge(closed, how='left', on=['project', 'key'])

In [31]:
full['key'].nunique()

8807

# last status update

In [32]:
ch = ch[~ch['created'].isnull()]
ch = ch[ch['field'] == 'status']
ch = ch[ch['fromString'] != ch['toString']]
ch 

last_status_update = ch.groupby(['project', 'key'], as_index=False)['created'].first()
last_status_update.columns = ['project', 'key', 'last_status_update_date']
last_status_update

,project,key,last_status_update_date
0,apstud,APSTUD-1313,2011-06-09 17:09:04+00:00
1,apstud,APSTUD-1378,2011-06-08 02:22:04+00:00
2,apstud,APSTUD-1469,2011-06-23 14:18:29+00:00
3,apstud,APSTUD-1507,2012-05-18 22:31:57+00:00
4,apstud,APSTUD-1698,2012-07-11 15:43:57+00:00
...,...,...,...
13914,xd,XD-995,2013-10-24 06:11:25.596000+00:00
13915,xd,XD-996,2014-03-25 03:39:32.012000+00:00
13916,xd,XD-997,2014-04-17 14:00:46.234000+00:00
13917,xd,XD-998,2013-10-25 08:32:20.511000+00:00


In [33]:
full = full.merge(last_status_update, how='left', on=['project', 'key'])

# velocity

In [34]:
def filter(x):
       # if ( (x['fields.resolutiondate'] <= x['sprint.endDate']) & 
       #     (x['fields.resolutiondate'] > x['sprint.startDate']) & 
       #     (x['fields.status.name'] in ['Done', 'Closed', 'Resolved', 'Accepted']) ): 
        
        if (x['project'] == 'mesos'):
            if (x['fields.status.name'] in ['Resolved']) & (x['last_status_update_date'] <= x['sprint.endDate']) & (x['last_status_update_date'] > x['sprint.startDate']):
                return True
            else:
                return False
        if (x['project'] == 'xd'):
            if (x['fields.status.name'] in ['Done']) & (x['last_status_update_date'] <= x['sprint.endDate']) & (x['last_status_update_date'] > x['sprint.startDate']):
                return True
            else:
                return False
        else:
            if  (x['fields.status.name'] in ['Closed']) & (x['last_status_update_date'] <= x['sprint.endDate']) & (x['last_status_update_date'] > x['sprint.startDate']):
                return True
            else:
                return False
            
#full_closed = full[full['closed_date'].notnull()]
    
full['velocity_flag'] = full.apply(lambda x : filter(x), axis=1)

In [35]:
full.groupby('project')['velocity_flag'].value_counts()

project  velocity_flag
apstud   True               301
         False              143
dnn      False            25888
         True              3855
mesos    False            31358
         True              2585
mule     False             2444
         True              1209
nexus    False             1955
         True               618
timob    False              577
         True               108
tistud   True              1409
         False              377
xd       True              1754
         False             1056
Name: velocity_flag, dtype: int64

In [36]:
full['key'].nunique()

8807

In [37]:
velocity = full[full['velocity_flag']].groupby(['project', 'sprint.name', 'sprint.startDate', 'sprint.endDate'])['storypoints'].sum()

In [38]:
velocity = velocity.reset_index().sort_values(['project', 'sprint.name'], ascending=True)

In [39]:
velocity.sort_values('sprint.startDate')
velocity.columns = ['project', 'sprint.name', 'startDate', 'endDate', 'velocity']
velocity

,project,sprint.name,startDate,endDate,velocity
0,apstud,2012 Sprint 02,2012-01-01 18:54:32.727272728+00:00,2012-01-27 00:00:00+00:00,90.0
1,apstud,2012 Sprint 03,2012-01-27 00:00:00+00:00,2012-02-10 00:00:00+00:00,228.0
2,apstud,2012 Sprint 04,2012-02-10 00:00:00+00:00,2012-02-24 00:00:00+00:00,168.0
3,apstud,2012 Sprint 05,2012-02-24 00:00:00+00:00,2012-03-09 00:00:00+00:00,127.0
4,apstud,2012 Sprint 06,2012-03-09 00:00:00+00:00,2012-03-23 00:00:00+00:00,121.0
...,...,...,...,...,...
475,xd,Sprint 67,2016-02-01 17:20:13.656000+00:00,2016-02-13 01:16:00+00:00,12.0
476,xd,Sprint 68,2016-02-16 00:38:45.289000+00:00,2016-02-26 08:34:00+00:00,7.0
477,xd,Sprint 7,2013-06-18 11:37:48.461000+00:00,2013-06-21 11:37:48.461000+00:00,35.0
478,xd,Sprint 8,2013-06-25 21:53:02.156000+00:00,2013-06-28 21:53:02.156000+00:00,150.0


# work capacity

for work capacity I have to calculate the sprint in which the issue count as "work" based on the last update date.

In [40]:
# sprints
sprints = full.groupby(['project', 'sprint.name'], as_index=False)[['sprint.startDate', 'sprint.endDate']].first()

sprints['sprint.startDate'] = pd.to_datetime(sprints['sprint.startDate'], utc=True)
sprints['sprint.endDate'] = pd.to_datetime(sprints['sprint.endDate'], utc=True)

sprints = sprints.sort_values(['project', 'sprint.endDate'], ascending=True)

In [41]:
# this covers the case in which the issue was updated in the same sprint to which was assigned
# full['last_update_sprint'] = full.apply(lambda x : x['sprint.name'] if (x['fields.updated'] <= x['sprint.endDate']) & (x['fields.updated'] > x['sprint.startDate']) else None, axis=1)

In [42]:
import datetime
import pytz

PROJECT = 'tistud'
DATE = datetime.datetime(2012, 12, 10, 0,0,0,0, pytz.UTC)

def check_sprint(x):
    # this covers the case in which the issue was updated in the same sprint to which was assigned
    # issues in this case should be part of velocity
    if (x['last_status_update_date'] <= x['sprint.endDate']) & (x['last_status_update_date'] > x['sprint.startDate']):
        return x['sprint.name'] 
    else:
        sp = sprints[sprints['project'] == x['project']]
        res = sp[(sp['sprint.endDate'] > x['last_status_update_date'])]
        if len(res) > 0:
            return res.head(1)['sprint.name'].values[0]
        else:
            return sp.tail(1)['sprint.name'].values[0]

full['last_update_sprint'] = full.apply(lambda x : check_sprint(x), axis=1)

In [43]:
full[full['last_update_sprint'].isna()]

,key,project,sprint.completeDate,sprint.endDate,sprint.id,sprint.name,sprint.startDate,sprint.state,fields.updated,fields.resolutiondate,storypoints,fields.status.name,fields.assignee.name,closed_date,last_status_update_date,velocity_flag,last_update_sprint


In [44]:
#def filter2(x):
#    if (x['fields.updated'] <= x['sprint.endDate']) & (x['closed_date'] > x['sprint.startDate']):
#        return True
#    else:
#        return False
    
#full['work_capacity_flag'] = full.apply(lambda x : filter2(x), axis=1)

In [45]:
work_capacity = full.groupby(['project', 'last_update_sprint'], as_index=False)['storypoints'].sum()
work_capacity.columns = ['project', 'sprint.name', 'work_capacity']
work_capacity

,project,sprint.name,work_capacity
0,apstud,2012 Sprint 02,232.0
1,apstud,2012 Sprint 03,371.0
2,apstud,2012 Sprint 04,170.0
3,apstud,2012 Sprint 05,163.0
4,apstud,2012 Sprint 06,173.0
...,...,...,...
536,xd,Sprint 67,18.0
537,xd,Sprint 68,22.0
538,xd,Sprint 7,45.0
539,xd,Sprint 8,242.0


In [46]:
metrics = velocity.merge(work_capacity, how='left', on=['project', 'sprint.name'])

In [47]:
metrics

,project,sprint.name,startDate,endDate,velocity,work_capacity
0,apstud,2012 Sprint 02,2012-01-01 18:54:32.727272728+00:00,2012-01-27 00:00:00+00:00,90.0,232.0
1,apstud,2012 Sprint 03,2012-01-27 00:00:00+00:00,2012-02-10 00:00:00+00:00,228.0,371.0
2,apstud,2012 Sprint 04,2012-02-10 00:00:00+00:00,2012-02-24 00:00:00+00:00,168.0,170.0
3,apstud,2012 Sprint 05,2012-02-24 00:00:00+00:00,2012-03-09 00:00:00+00:00,127.0,163.0
4,apstud,2012 Sprint 06,2012-03-09 00:00:00+00:00,2012-03-23 00:00:00+00:00,121.0,173.0
...,...,...,...,...,...,...
475,xd,Sprint 67,2016-02-01 17:20:13.656000+00:00,2016-02-13 01:16:00+00:00,12.0,18.0
476,xd,Sprint 68,2016-02-16 00:38:45.289000+00:00,2016-02-26 08:34:00+00:00,7.0,22.0
477,xd,Sprint 7,2013-06-18 11:37:48.461000+00:00,2013-06-21 11:37:48.461000+00:00,35.0,45.0
478,xd,Sprint 8,2013-06-25 21:53:02.156000+00:00,2013-06-28 21:53:02.156000+00:00,150.0,242.0


# focus factor

velocity / work_capacity

In [48]:
metrics['focus_factor'] = metrics['velocity'] / metrics['work_capacity']
metrics['sprint_length'] = pd.to_datetime(metrics['endDate']) - pd.to_datetime(metrics['startDate'])
metrics

,project,sprint.name,startDate,endDate,velocity,work_capacity,focus_factor,sprint_length
0,apstud,2012 Sprint 02,2012-01-01 18:54:32.727272728+00:00,2012-01-27 00:00:00+00:00,90.0,232.0,0.387931,25 days 05:05:27.272727
1,apstud,2012 Sprint 03,2012-01-27 00:00:00+00:00,2012-02-10 00:00:00+00:00,228.0,371.0,0.614555,14 days 00:00:00
2,apstud,2012 Sprint 04,2012-02-10 00:00:00+00:00,2012-02-24 00:00:00+00:00,168.0,170.0,0.988235,14 days 00:00:00
3,apstud,2012 Sprint 05,2012-02-24 00:00:00+00:00,2012-03-09 00:00:00+00:00,127.0,163.0,0.779141,14 days 00:00:00
4,apstud,2012 Sprint 06,2012-03-09 00:00:00+00:00,2012-03-23 00:00:00+00:00,121.0,173.0,0.699422,14 days 00:00:00
...,...,...,...,...,...,...,...,...
475,xd,Sprint 67,2016-02-01 17:20:13.656000+00:00,2016-02-13 01:16:00+00:00,12.0,18.0,0.666667,11 days 07:55:46.344000
476,xd,Sprint 68,2016-02-16 00:38:45.289000+00:00,2016-02-26 08:34:00+00:00,7.0,22.0,0.318182,10 days 07:55:14.711000
477,xd,Sprint 7,2013-06-18 11:37:48.461000+00:00,2013-06-21 11:37:48.461000+00:00,35.0,45.0,0.777778,3 days 00:00:00
478,xd,Sprint 8,2013-06-25 21:53:02.156000+00:00,2013-06-28 21:53:02.156000+00:00,150.0,242.0,0.619835,3 days 00:00:00


In [49]:
assert np.all(metrics['focus_factor'] >= 0)
assert np.all(metrics['focus_factor'] <= 1)
assert np.all(metrics['work_capacity'] >= metrics['velocity'])

# baseline

In [50]:
metrics = metrics.sort_values(['project', 'startDate'], ascending=True)
baseline = metrics.groupby(['project'], as_index=False)[['project','sprint.name','velocity']].head(3)
baseline = baseline.groupby(['project'])['velocity'].mean()
baseline.name = 'baseline'
baseline

project
apstud    162.000000
dnn       375.633333
mesos      59.000000
mule       26.000000
nexus      19.666667
timob      10.666667
tistud    106.666667
xd          8.000000
Name: baseline, dtype: float64

In [51]:
metrics = metrics.merge(baseline, on='project')

# velocity increase

In [52]:
metrics['velocity_increase'] = metrics['velocity'] / metrics['baseline']

# hyperproductivity

\begin{equation}
P_i = \begin{cases}
        \textrm{Hyper-productive} & \text{if}\;  V_i \ge 400\% \\
        \textrm{Proficient} & \textrm{if}\; 240\% \le V_i < 400\% \\
        \textrm{Acceptable} & \textrm{if}\; 100\% \le V_i < 240\% \\
        \textrm{Low-performance} & \textrm{if}\; 0\% \le V_i < 100\% \\
    \end{cases}
\label{eq:hyper-productivity}
\end{equation}

In [53]:
def hyper_classifier(v):
    """
    4 = hyper
    3 = proficient
    2 = acceptable
    1 = low
    """
    if v >= 4:
        return 4
    if (v >= 2.4) & (v < 4):
        return 3
    if (v >= 1) & (v < 2.4):
        return 2
    if (v < 1):
        return 1

In [54]:
metrics['P'] = metrics['velocity_increase'].apply(lambda x : hyper_classifier(x))

# healthy

\begin{equation}
    H_i = \begin{cases}
     \textrm{Healthy} & \textrm{if}\; 70\% \le F_i \le 90\% \\
     \textrm{Unhealthy} & \textrm{otherwise}\;
    \end{cases}
    \label{eq:health-state}
\end{equation}

In [55]:
def healthy_classifier(f):
    if (f >= .7) & (f <= .9):
        return 1
    else:
        return 0

In [56]:
metrics['H'] = metrics['focus_factor'].apply(lambda x : healthy_classifier(x))

In [57]:
metrics['project'].unique()

array(['apstud', 'dnn', 'mesos', 'mule', 'nexus', 'timob', 'tistud', 'xd'],
      dtype=object)

In [58]:
metrics

,project,sprint.name,startDate,endDate,velocity,work_capacity,focus_factor,sprint_length,baseline,velocity_increase,P,H
0,apstud,2012 Sprint 02,2012-01-01 18:54:32.727272728+00:00,2012-01-27 00:00:00+00:00,90.0,232.0,0.387931,25 days 05:05:27.272727,162.0,0.555556,1,0
1,apstud,2012 Sprint 03,2012-01-27 00:00:00+00:00,2012-02-10 00:00:00+00:00,228.0,371.0,0.614555,14 days 00:00:00,162.0,1.407407,2,0
2,apstud,2012 Sprint 04,2012-02-10 00:00:00+00:00,2012-02-24 00:00:00+00:00,168.0,170.0,0.988235,14 days 00:00:00,162.0,1.037037,2,0
3,apstud,2012 Sprint 05,2012-02-24 00:00:00+00:00,2012-03-09 00:00:00+00:00,127.0,163.0,0.779141,14 days 00:00:00,162.0,0.783951,1,1
4,apstud,2012 Sprint 06,2012-03-09 00:00:00+00:00,2012-03-23 00:00:00+00:00,121.0,173.0,0.699422,14 days 00:00:00,162.0,0.746914,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
475,xd,Sprint 62,2015-11-16 16:10:26.278000+00:00,2015-11-27 17:06:00+00:00,6.0,11.0,0.545455,11 days 00:55:33.722000,8.0,0.750000,1,0
476,xd,Sprint 63,2015-11-30 16:32:01.258000+00:00,2015-12-11 17:28:00+00:00,12.0,16.0,0.750000,11 days 00:55:58.742000,8.0,1.500000,2,1
477,xd,Sprint 64,2015-12-17 18:20:12.425000+00:00,2015-12-26 02:16:00+00:00,1.0,1.0,1.000000,8 days 07:55:47.575000,8.0,0.125000,1,0
478,xd,Sprint 67,2016-02-01 17:20:13.656000+00:00,2016-02-13 01:16:00+00:00,12.0,18.0,0.666667,11 days 07:55:46.344000,8.0,1.500000,2,0


# developers

In [59]:
full.groupby(['project', 'sprint.name'])['fields.assignee.name'].nunique()

project  sprint.name   
apstud   2012 Sprint 02    3
         2012 Sprint 03    4
         2012 Sprint 04    5
         2012 Sprint 05    4
         2012 Sprint 06    5
                          ..
xd       Sprint 67         2
         Sprint 68         4
         Sprint 7          7
         Sprint 8          9
         Sprint 9          7
Name: fields.assignee.name, Length: 617, dtype: int64

In [60]:
devs = full.groupby(['project', 'sprint.name'])['fields.assignee.name'].unique()
devs = pd.DataFrame(devs)
devs = devs.reset_index()

In [61]:
devs

,project,sprint.name,fields.assignee.name
0,apstud,2012 Sprint 02,"[mxia, cwilliams, klindsey]"
1,apstud,2012 Sprint 03,"[sgibly, mxia, klindsey, cwilliams]"
2,apstud,2012 Sprint 04,"[cwilliams, klindsey, ingo, mxia, sgibly]"
3,apstud,2012 Sprint 05,"[cwilliams, klindsey, sgibly, mxia]"
4,apstud,2012 Sprint 06,"[cwilliams, klindsey, sgibly, ingo, mxia]"
...,...,...,...
612,xd,Sprint 67,"[grussell, hillert, nan]"
613,xd,Sprint 68,"[grussell, abilan, dturanski, nan, iperumal]"
614,xd,Sprint 7,"[grussell, eric.bottard, luke, mark.fisher, hi..."
615,xd,Sprint 8,"[thomas.risberg, iperumal, grenfro, mark.fishe..."


In [62]:
sprint_devs = []
for p in devs.project.unique():
    pdevs = devs[devs.project == p].copy()
    
    pdevs.loc[:,'old_devs'] = pdevs.shift(1)['fields.assignee.name'].copy()
    
    pdevs.loc[:,'old_devs_abs'] = pdevs[1:].apply(lambda x : len(set(x['old_devs'])) , axis=1)
    pdevs.loc[:,'new_devs_abs'] = pdevs[1:].apply(lambda x : len(set(x['fields.assignee.name']) - set(x['old_devs'])) , axis=1)

    pdevs.loc[:,'turnover_abs'] = pdevs[1:].apply(lambda x : len(set(x['old_devs']) - set(x['fields.assignee.name'])) , axis=1)

    # the first one is nan since there are no old devs at the very begining. then I assign 0
    pdevs.loc[:,'new_devs_abs'] = pdevs['new_devs_abs'].fillna(0)
    pdevs.loc[:,'turnover_abs'] = pdevs['turnover_abs'].fillna(0)
    pdevs.loc[:,'old_devs_abs'] = pdevs['old_devs_abs'].fillna(0)
    
    pdevs.loc[:,'current_devs_abs'] = pdevs['fields.assignee.name'].apply(lambda x : len(x))
    
    pdevs.loc[:,'turnover'] = pdevs['turnover_abs'] / (pdevs['old_devs_abs'] + pdevs['current_devs_abs'])/2
    #pdevs['new_devs'] = pdevs['new_devs_abs'] / pdevs['current_devs_abs']
    pdevs.loc[:,'new_devs'] = pdevs['new_devs_abs'] / (pdevs['old_devs_abs'] + pdevs['current_devs_abs'])/2
    
    sprint_devs.append(pdevs)
    
sprint_devs = pd.concat(sprint_devs, axis=0)

In [63]:
sprint_devs

,project,sprint.name,fields.assignee.name,old_devs,old_devs_abs,new_devs_abs,turnover_abs,current_devs_abs,turnover,new_devs
0,apstud,2012 Sprint 02,"[mxia, cwilliams, klindsey]",NaN,0.0,0.0,0.0,3,0.000000,0.000000
1,apstud,2012 Sprint 03,"[sgibly, mxia, klindsey, cwilliams]","[mxia, cwilliams, klindsey]",3.0,1.0,0.0,4,0.000000,0.071429
2,apstud,2012 Sprint 04,"[cwilliams, klindsey, ingo, mxia, sgibly]","[sgibly, mxia, klindsey, cwilliams]",4.0,1.0,0.0,5,0.000000,0.055556
3,apstud,2012 Sprint 05,"[cwilliams, klindsey, sgibly, mxia]","[cwilliams, klindsey, ingo, mxia, sgibly]",5.0,0.0,1.0,4,0.055556,0.000000
4,apstud,2012 Sprint 06,"[cwilliams, klindsey, sgibly, ingo, mxia]","[cwilliams, klindsey, sgibly, mxia]",4.0,1.0,0.0,5,0.000000,0.055556
...,...,...,...,...,...,...,...,...,...,...
612,xd,Sprint 67,"[grussell, hillert, nan]",[grussell],1.0,2.0,0.0,3,0.000000,0.250000
613,xd,Sprint 68,"[grussell, abilan, dturanski, nan, iperumal]","[grussell, hillert, nan]",3.0,3.0,1.0,5,0.062500,0.187500
614,xd,Sprint 7,"[grussell, eric.bottard, luke, mark.fisher, hi...","[grussell, abilan, dturanski, nan, iperumal]",5.0,6.0,4.0,7,0.166667,0.250000
615,xd,Sprint 8,"[thomas.risberg, iperumal, grenfro, mark.fishe...","[grussell, eric.bottard, luke, mark.fisher, hi...",7.0,4.0,2.0,9,0.062500,0.125000


In [64]:
metrics = metrics.merge(sprint_devs, on=['project', 'sprint.name'])

In [65]:
metrics.shape

(480, 20)

# summary

In [66]:
pd.concat([metrics.pivot_table(index='project', columns=['P'], values='sprint.name', aggfunc='nunique'),
           metrics.pivot_table(index='project', columns=['H'], values='sprint.name', aggfunc='nunique'),
           metrics.groupby('project')['sprint.name'].count(),
          ], axis=1)

,1,2,3,4,0,1,sprint.name
project,,,,,,,
apstud,19.0,6.0,NaN,NaN,16.0,9.0,25
dnn,78.0,7.0,NaN,NaN,83.0,2.0,87
mesos,27.0,23.0,6.0,9.0,64.0,1.0,65
mule,34.0,21.0,14.0,25.0,78.0,16.0,94
nexus,55.0,8.0,NaN,NaN,62.0,1.0,63
timob,12.0,7.0,NaN,3.0,22.0,NaN,22
tistud,22.0,32.0,4.0,NaN,24.0,34.0,58
xd,4.0,5.0,6.0,51.0,47.0,19.0,66


candidates are: mesos, mule, xd

In [67]:
metrics.groupby('project').describe().T

project            apstud        dnn      mesos       mule      nexus  \
velocity count         25         87         65         94         63   
         mean       95.64    101.289    184.954    71.0957    8.51587   
         std      82.9502    161.073    571.746    69.9625    8.74088   
         min            5          1          3          2        0.5   
         25%           20          5         27         16          2   
...                   ...        ...        ...        ...        ...   
new_devs min            0          0          0          0          0   
         25%            0          0  0.0277778          0  0.0442308   
         50%         0.05  0.0833333  0.0555556  0.0833333  0.0909091   
         75%    0.0714286   0.166667        0.1   0.166667   0.166667   
         max          0.2        0.4   0.333333        0.4        0.4   

project           timob     tistud         xd  
velocity count       22         58         66  
         mean   17.4318    137.276    90.7758  
         std    21.1993    70.2763    84.6127  
         min        0.5         18          1  
         25%          6      93.25      36.75  
...                 ...        ...        ...  
new_devs min          0          0          0  
         25%       0.25          0  0.0266813  
         50%     0.3125          0  0.0588235  
         75%    0.39375  0.0454545   0.112401  
         max     0.4375        0.1    0.28125  

[112 rows x 8 columns]

# add sprint number

In [68]:
l = []

for p in metrics.project.unique():
    m = metrics[metrics.project == p]
    print(p, len(m))
    
    m['sprint.nbr'] = range(1,len(m)+1)
    l.append(m)

metrics = pd.concat(l, axis=0)

apstud 25
dnn 87
mesos 65
mule 94
nexus 63
timob 22
tistud 58
xd 66


C:\Users\Ezequiel\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [69]:
metrics

,project,sprint.name,startDate,endDate,velocity,work_capacity,focus_factor,sprint_length,baseline,velocity_increase,...,H,fields.assignee.name,old_devs,old_devs_abs,new_devs_abs,turnover_abs,current_devs_abs,turnover,new_devs,sprint.nbr
0,apstud,2012 Sprint 02,2012-01-01 18:54:32.727272728+00:00,2012-01-27 00:00:00+00:00,90.0,232.0,0.387931,25 days 05:05:27.272727,162.0,0.555556,...,0,"[mxia, cwilliams, klindsey]",NaN,0.0,0.0,0.0,3,0.000000,0.000000,1
1,apstud,2012 Sprint 03,2012-01-27 00:00:00+00:00,2012-02-10 00:00:00+00:00,228.0,371.0,0.614555,14 days 00:00:00,162.0,1.407407,...,0,"[sgibly, mxia, klindsey, cwilliams]","[mxia, cwilliams, klindsey]",3.0,1.0,0.0,4,0.000000,0.071429,2
2,apstud,2012 Sprint 04,2012-02-10 00:00:00+00:00,2012-02-24 00:00:00+00:00,168.0,170.0,0.988235,14 days 00:00:00,162.0,1.037037,...,0,"[cwilliams, klindsey, ingo, mxia, sgibly]","[sgibly, mxia, klindsey, cwilliams]",4.0,1.0,0.0,5,0.000000,0.055556,3
3,apstud,2012 Sprint 05,2012-02-24 00:00:00+00:00,2012-03-09 00:00:00+00:00,127.0,163.0,0.779141,14 days 00:00:00,162.0,0.783951,...,1,"[cwilliams, klindsey, sgibly, mxia]","[cwilliams, klindsey, ingo, mxia, sgibly]",5.0,0.0,1.0,4,0.055556,0.000000,4
4,apstud,2012 Sprint 06,2012-03-09 00:00:00+00:00,2012-03-23 00:00:00+00:00,121.0,173.0,0.699422,14 days 00:00:00,162.0,0.746914,...,0,"[cwilliams, klindsey, sgibly, ingo, mxia]","[cwilliams, klindsey, sgibly, mxia]",4.0,1.0,0.0,5,0.000000,0.055556,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,xd,Sprint 62,2015-11-16 16:10:26.278000+00:00,2015-11-27 17:06:00+00:00,6.0,11.0,0.545455,11 days 00:55:33.722000,8.0,0.750000,...,0,"[mminella, mbogoevici, grussell, jvalkeal, hil...","[grussell, eric.bottard, hillert, thomas.risbe...",8.0,3.0,3.0,8,0.093750,0.093750,62
476,xd,Sprint 63,2015-11-30 16:32:01.258000+00:00,2015-12-11 17:28:00+00:00,12.0,16.0,0.750000,11 days 00:55:58.742000,8.0,1.500000,...,1,"[eric.bottard, jvalkeal, grussell, thomas.risb...","[mminella, mbogoevici, grussell, jvalkeal, hil...",8.0,1.0,5.0,4,0.208333,0.041667,63
477,xd,Sprint 64,2015-12-17 18:20:12.425000+00:00,2015-12-26 02:16:00+00:00,1.0,1.0,1.000000,8 days 07:55:47.575000,8.0,0.125000,...,0,[grussell],"[eric.bottard, jvalkeal, grussell, thomas.risb...",4.0,0.0,3.0,1,0.300000,0.000000,64
478,xd,Sprint 67,2016-02-01 17:20:13.656000+00:00,2016-02-13 01:16:00+00:00,12.0,18.0,0.666667,11 days 07:55:46.344000,8.0,1.500000,...,0,"[grussell, hillert, nan]",[grussell],1.0,2.0,0.0,3,0.000000,0.250000,65


In [70]:
metrics['sprint_length'] = metrics['sprint_length'].dt.days

In [71]:
metrics['velocity_increase'] = metrics['velocity_increase'] * 100

# team stability index

In [72]:
for p in metrics['project'].unique():
    d = metrics[metrics['project'] == p]
    
    initial = d.loc[d['sprint.nbr'] == 1, 'current_devs_abs'].values[0]

    metrics.loc[d.index, 'TSI'] = (initial + d['new_devs_abs'] + d['turnover_abs']) / initial

In [73]:
metrics['TSI_inv'] = 1 / metrics['TSI']

# individual velocity

In [74]:
metrics['individual_velocity'] = metrics['velocity'] / metrics['current_devs_abs']

# save

In [75]:
metrics.to_csv('data/metrics_iterations.csv', index=False)
full.to_csv('data/metrics_issues.csv', index=False)